In [3]:
import pandas as pd

In [4]:
import solver, submit
reload(solver)

<module 'solver' from 'solver.pyc'>

In [5]:
from collections import namedtuple
from operator import attrgetter
Item = namedtuple("Item", ['index', 'value', 'weight','value_weight'])

In [6]:
def load_input_data(filename):
    input_data = submit.load_input_data(filename)
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1]), float(parts[0])/float(parts[1])))
    return capacity, sorted(items, key=attrgetter('value_weight'), reverse=True)

In [7]:
import math
import numpy as np

In [8]:
def compute_solution(capacity,items,nb_points,lb_flag):
    opt, check, takens = solver.dyn_prog_solve_v2(capacity,items,nb_points,lb_flag)
    print 'opt %s: %.f check: %.f/%.f (perf : %.2f%s) '%('lb' if lb_flag else 'ub',
                                                         opt,
                                                     check, 
                                                     capacity,
                                                     float(check)/capacity*100,'%')+str(takens)
    return opt, takens

In [130]:
capacity, items = load_input_data('./data/ks_10000_0')

In [54]:
solver.heuristic(capacity,items)

134007.03715926342

In [55]:
capacity

100000

In [91]:
%time opt1_lb, takens1_lb = compute_solution(capacity,items,1000,True)

opt lb: 3960932 check: 9471864/9486367 (perf : 99.85%) [389, 237, 375, 393, 391, 376]
Wall time: 2.17 s


In [57]:
%time opt2_lb, takens2_lb = compute_solution(capacity,items,10000,True)

opt lb: 109847 check: 99957/100000 (perf : 99.96%) [124, 627, 640, 703, 485, 504, 115, 525, 553, 336, 519, 642]
Wall time: 52.5 s


In [58]:
solver.check_knapsack(capacity,items,takens1_lb,-1)

(True, 0.0, 109493)

In [59]:
solver.check_knapsack(capacity,items,takens2_lb,-1)

(True, 0.0, 109847)

In [92]:
items

[Item(index=374, value=1365820, weight=3265540, value_weight=0.41825241767058435),
 Item(index=376, value=1445900, weight=3457100, value_weight=0.4182407219924214),
 Item(index=391, value=1417728, weight=3389756, value_weight=0.41823895289218455),
 Item(index=399, value=1211040, weight=2895580, value_weight=0.4182374515641081),
 Item(index=379, value=492496, weight=1177692, value_weight=0.4181874378020739),
 Item(index=377, value=501060, weight=1198220, value_weight=0.41817028592412075),
 Item(index=393, value=491820, weight=1176140, value_weight=0.4181645042256874),
 Item(index=375, value=562272, weight=1344644, value_weight=0.41815677606860996),
 Item(index=174, value=387480, weight=926660, value_weight=0.41814689314311615),
 Item(index=334, value=387480, weight=926660, value_weight=0.41814689314311615),
 Item(index=52, value=395124, weight=944948, value_weight=0.41814364388304964),
 Item(index=397, value=400156, weight=957012, value_weight=0.4181305981534192),
 Item(index=380, value

In [46]:
def nb_errors(capacity, items, takens):
    total_weight = 0
    total_value = 0
    error = 0
    for item in items:
        if total_weight + item.weight < capacity:
            if item.index not in takens:
                error += 1
            else:
                total_weight += item.weight
                total_value += item.value
    return error, total_weight, total_value

In [66]:
nb_errors(capacity,items,takens3)

(1, 99999, 109899)

In [ ]:
reload(solver)

In [61]:
import sys

In [62]:
sys.setrecursionlimit(2000)
sys.getrecursionlimit()

2000

In [105]:
items[:20]

[Item(index=374, value=1365820, weight=3265540, value_weight=0.41825241767058435),
 Item(index=376, value=1445900, weight=3457100, value_weight=0.4182407219924214),
 Item(index=391, value=1417728, weight=3389756, value_weight=0.41823895289218455),
 Item(index=399, value=1211040, weight=2895580, value_weight=0.4182374515641081),
 Item(index=379, value=492496, weight=1177692, value_weight=0.4181874378020739),
 Item(index=377, value=501060, weight=1198220, value_weight=0.41817028592412075),
 Item(index=393, value=491820, weight=1176140, value_weight=0.4181645042256874),
 Item(index=375, value=562272, weight=1344644, value_weight=0.41815677606860996),
 Item(index=174, value=387480, weight=926660, value_weight=0.41814689314311615),
 Item(index=334, value=387480, weight=926660, value_weight=0.41814689314311615),
 Item(index=52, value=395124, weight=944948, value_weight=0.41814364388304964),
 Item(index=397, value=400156, weight=957012, value_weight=0.4181305981534192),
 Item(index=380, value

In [123]:
%time opt3,takens3 = solver.banch_bound_depth_first(capacity,items,0)
opt3, takens3

Wall time: 159 ms


(3967180, [374, 376, 377, 375, 107, 164])

In [121]:
nb_errors(capacity, items,takens3)

(2, 9486360, 3967180)

In [65]:
solver.check_knapsack(capacity,items,takens3,-1)

(True, 0.0, 109899)

In [124]:
def branch_bound_depth_first_v2(capacity, items, max_error, error_index= []):
    if max_error < 0:
        return 0, 0, []
    
    total_weight = 0
    total_value = 0
    selected_index = []
    for item in items:
        if item.index not in error_index:
            if total_weight + item.weight < capacity:
                selected_index += [item.index]
                total_weight += item.weight
                total_value += item.value
                
    for idx in selected_index:
        total_value_2, total_weight_2, selected_index_2 = branch_bound_depth_first_v2(capacity,
                                                                      items,
                                                                      max_error-1,
                                                                      error_index+[idx])
        if total_value_2>total_value:
            total_value = total_value_2
            total_weight = total_weight_2
            selected_index = selected_index_2
    
    return total_value, total_weight, selected_index

In [136]:
branch_bound_depth_first_v2(capacity,items,1)

(1099893,
 999994,
 [3003,
  568,
  7055,
  3023,
  8034,
  9756,
  1824,
  7577,
  2946,
  2641,
  9431,
  2192,
  2827,
  7498,
  6113])